## Final Run—Monopoly Game Agent

The following notebook creates agents to automatically play Monopoly. You can
specify the behaviour of these agents in two ways:
- How the agents decide whether to buy a property
- How the agents decide whether to build a house/hotel on a property

At present, more extensive decision-making features (like trading or mortgaging) are not included. However, these could be added in future versions.

### Project Setup

In [8]:
%pip install -r requirements.txt


[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Import Dependencies

In [9]:
# monopoly simulator
from simulator.monosim.player import Player
from simulator.monosim.board import get_board, get_roads, get_properties, get_community_chest_cards, get_bank

# LangChain
from langchain_openai import ChatOpenAI
from langchain import OpenAI, LLMChain, PromptTemplate

# other imports
from pydantic import BaseModel, Field
from typing import List

# dotenv for loading API keys
import os
from dotenv import load_dotenv

#Python 3.9.6
from typing import Union

In [10]:
# Load API keys from .env file
load_dotenv()

# Define the API keys
openai_key = os.getenv("OPENAI_API_KEY")

### Courtroom Architecture

The following three chains will generate arguments for, against, and a final decision.

In [11]:
# Lawyer Pro: Argument for buying the property
pro_prompt = PromptTemplate(
    input_variables=["property_details", "player_stats"], # can prolly add past_cases here
    template="""
    You are a lawyer arguing FOR the decision to buy a property in Monopoly.

    Property Details:
    {property_details}

    Player Stats:
    {player_stats}

    Build your argument FOR why buying this property is a strategic decision to maximize winning chances.
    """
    # Can add the following later.
    # Past Relevant Cases:
    # {past_cases}
)
pro_chain = LLMChain(llm=ChatOpenAI(model="gpt-4o-mini"), prompt=pro_prompt)

# Lawyer Con: Argument against buying the property
con_prompt = PromptTemplate(
    input_variables=["property_details", "player_stats"], # can prolly add past_cases here
    template="""
    You are a lawyer arguing AGAINST the decision to buy a property in Monopoly.

    Property Details:
    {property_details}

    Player Stats:
    {player_stats}

    Build your argument AGAINST why buying this property is a strategic decision.

    Can add the following later.
    Past Relevant Cases:
    
    {past_cases}
    """
)
con_chain = LLMChain(llm=ChatOpenAI(model="gpt-4o-mini"), prompt=con_prompt)

# Judge: Final decision based on arguments
judge_prompt = PromptTemplate(
    input_variables=["property_details", "player_stats", "arguments_for", "arguments_against"],
    template="""
    You are a judge deciding whether a player should buy a property in Monopoly. Your goal is to maximize the player's chances of winning the game with forward-thinking reasoning.
    
    Property Details:
    {property_details}

    Player Stats:
    {player_stats}

    Argument FOR buying the property:
    {arguments_for}

    Argument AGAINST buying the property:
    {arguments_against}

    Make your final decision: Should the player buy this property? Justify your reasoning briefly.
    Output "buy" or "not buy" as your final decision.
    """
)
judge_chain = LLMChain(llm=ChatOpenAI(model="gpt-4o-mini"), prompt=judge_prompt)

### Vectorstore

In [12]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document

vectorstore_path = "vectorstore"

embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")

try:
    vectorstore = FAISS.load_local(vectorstore_path, embedding_model, allow_dangerous_deserialization=True)

except RuntimeError:
    print("Vectorstore not found. Initializing a new one.")

    dummy_doc = [Document(page_content="Temporary initialization document.")]
    
    vectorstore = FAISS.from_documents(dummy_doc, embedding_model)  

    vectorstore.save_local(vectorstore_path)  


def retrieve_similar_cases(context: str, k: int = 3):
    """
    Retrieve similar cases from the vectorstore based on the provided context.
    """
    results = vectorstore.similarity_search(context, k=k)
    return results if results else []

def add_case_to_vectorstore(case_id: str, context: str, arguments: str, decision: str):
    """
    Add a new case to the vectorstore for future retrieval.
    """
    new_case = Document(
        page_content=f"Context: {context}\nArguments: {arguments}\nDecision: {decision}",
        metadata={"case_id": case_id}
    )
    vectorstore.add_documents([new_case])
    vectorstore.save_local(vectorstore_path)  # Save updates locally

### Buying Properties
This function can be modified to get the agent to behave differently when deciding whether to buy a property. The placeholders are just examples of what you can do. Note: you don't need to use all the information available when making a decision.


In [14]:
def custom_buy(self, dict_property_info : dict) -> Union[str, None]:
    """
    Edit this function to determine whether the player should buy a property. 

    Return:
    YOUR FUNCTION MUST RETURN "buy" (type str) to buy the property 
    or None (type None) to not buy the property.
    
    WARNING: I've included exhaustive documentation on all the data you can 
    use to make this decision. DON'T use all of it; it's mostly irrelevant. 

    Parameters:
    self: useful for accessing the following methods
    - self.get_state(): returns player data in a dict with the keys
        - cash: the amount of cash the player has
        - owned_roads: a list of owned roads
        - owned_stations: a list of owned stations
        - owned_utilities: a list of owned utilities
        - owned_colors: a dict which maps colors to True if the player owns all 
            properties of that color. Ex: owned_colors['blue'] = True/False
        - owned_houses_hotels: a dict which maps property names to the number of 
            houses/hotels built there. Ex: owned_houses_hotels['Park Place'] = 3
        - mortgageable_amount: total mortgageable value of player's properties
        WARNING: PROBABLY NOT USEFUL DATA BELOW
        - morgaged_roads: a list of mortgaged roads
        - mortgaged_stations: a list of mortgaged stations
        - mortgaged_utilities: a list of mortgaged utilities
        - name: name of player
        - number: player number
        - position: player's position on the board
        - dice_value: the sum of the dice rolled
        - jail_count: the number of turns the player has been in jail
        - exit_jail: whether the player has exited jail
        - free_visit: whether the player is visiting jail for free
        - has_lost: whether the player has lost the game
    - self._dict_properties[property_name]: returns property info for an 
    arbitrary property with a dict like dict_property_info below

    dict_property_info: Has a dict with info about the property to buy/not buy. 
        The dict has the following keys:
    * name: the name of the property
    * belongs_to: the player who owns the property
    * price: the price of the property
    * rent: the rent of the property
    - color: the color of the property
    - rent_with_color_set: rent when all properties of same color are owned
    * type: the type of property (one of 'road', 'station', 'utility')
    - houses_cost: the cost of building a house
    - hotels_cost: the cost of building a hotel
    - rent_with_X_houses_0_hotels: rent when X = {1, 2, 3, 4} houses are built
    - rent_with_4_houses_1_hotels: rent when 4 houses and 1 hotel are built
    WARNING: PROBABLY NOT USEFUL DATA BELOW
    * mortgage_value: the amount of money earned when mortgaging the property
    * unmortgage_value: the amount of money cost to unmortgage the property
    * is_mortgaged: whether the property is mortgaged
    * board_num: the position of the property on the board
    P.S. (* means for all property types, - means for road properties only)
    """

    # currently, no trading. So don't buy property if someone else owns it
    if dict_property_info['belongs_to'] != None:
        return None

    # If the property isn't owned, I get the LLM to decide whether to buy it
    # Steps: 1. making a prompt template, 2. injecting data, 3. setting up 
    # output parsing, and 4. running an LLM.

    ##################################################
    # Step 1: Gather game state and property data
    ##################################################
    property_details = f"""
    Property Name: {dict_property_info['name']}
    Type: {dict_property_info['type']}
    Cost: {dict_property_info['price']}
    Base Rent: {dict_property_info['rent']}
    """



    # Add specific details for each property type
    if dict_property_info['type'] == 'road':
        # Calculate number of properties of the same color owned
        n_color_properties = 0
        for property_name in self.get_state()['owned_roads']:
            property_name = property_name.strip().lower()  # normalize name
            prop_info = self._dict_properties.get(property_name)  # safe access
            if prop_info and prop_info['color'] == dict_property_info['color']:
                n_color_properties += 1

        property_details += f"""
        Color: {dict_property_info['color']}
        Cost of Building a House: {dict_property_info['houses_cost']}
        Cost of Building a Hotel: {dict_property_info['hotels_cost']}
        Rent with 4 Houses and 1 Hotel: {dict_property_info['rent_with_4_houses_1_hotels']}
        Number of {dict_property_info['color']} properties owned: {n_color_properties}
        """
    elif dict_property_info['type'] == 'station':
        property_details += f"""
        Number of Stations Owned: {len(self.get_state()['owned_stations'])}
        """
    elif dict_property_info['type'] == 'utility':
        property_details += f"""
        Number of Utilities Owned: {len(self.get_state()['owned_utilities'])}
        """

    player_stats = f"""
    Cash: {self.get_state()['cash']}
    Owned Roads: {len(self.get_state()['owned_roads'])}
    Owned Stations: {len(self.get_state()['owned_stations'])}
    Owned Utilities: {len(self.get_state()['owned_utilities'])}
    """

    # ##################################################
    # # Step 2: Retrieve past cases
    # ##################################################
    past_cases = retrieve_similar_cases(context=f"{property_details} {player_stats}", k=3)
    past_cases_text = "\n".join([case.page_content for case in past_cases])

    ##################################################
    # Step 3: Lawyer Pro and Lawyer Con Arguments
    ##################################################
    arguments_for = pro_chain.run(
        property_details=property_details,
        player_stats=player_stats,
        past_cases=past_cases_text
    )

    arguments_against = con_chain.run(
        property_details=property_details,
        player_stats=player_stats,
        past_cases=past_cases_text
    )

    ##################################################
    # Step 4: Judge's Decision
    ##################################################
    decision = judge_chain.run(
        property_details=property_details,
        player_stats=player_stats,
        arguments_for=arguments_for,
        arguments_against=arguments_against
    )

    # ##################################################
    # # Step 5: Store the decision in the vectorstore
    # ##################################################
    add_case_to_vectorstore(
        case_id=f"{dict_property_info['name']}_{self.get_state()['name']}",
        context=f"{property_details} {player_stats}",
        arguments=f"For: {arguments_for}\nAgainst: {arguments_against}",
        decision=decision
    )

    ##################################################
    # Step 6: Return Decision
    ##################################################
    # for us to see the output
    print(f"Lawyer FOR Arguments: {arguments_for}\n\n")
    print(f"Laywer AGAINST Arguments: {arguments_against}\n\n")
    print(f"Judge's Decision: {decision}\n\n\n\n")
    return "buy" if "buy" in decision.lower() else None

### Backend Setup

In [15]:
def modify_buy_or_bid(buy) -> str:
    return custom_buy

class CustomPlayer(Player):
    buy_or_bid = modify_buy_or_bid(Player.buy_or_bid)

In [16]:
def initialize_game() -> dict:
    """
    Initializes a game with two players and sets up the bank, board, roads, properties, 
    and community chest cards.
    
    Returns:
        dict: A dictionary containing the following:
            - "bank": Game's bank object.
            - "board": Main game board.
            - "roads": List of road objects.
            - "properties": List of property objects.
            - "community_chest_cards": Dictionary of community chest cards.
            - "players": List of two Player objects, with Player 1 first.
    """
    
    bank = get_bank()
    board = get_board()
    roads = get_roads()
    properties = get_properties()
    community_chest_cards = get_community_chest_cards()
    community_cards_deck = list(community_chest_cards.keys())

    # Note how we have one of our players vs. a default player that just buys 
    # whenever cash is available. We can change this later
    player1 = CustomPlayer('Alice', 1, bank, board, roads, properties, community_cards_deck)
    player2 = Player('Bob', 2, bank, board, roads, properties, community_cards_deck)
    
    player1.meet_other_players([player2])
    player2.meet_other_players([player1])
    
    return {
        "bank": bank,
        "board": board,
        "roads": roads,
        "properties": properties,
        "community_chest_cards": community_chest_cards,
        "players": [player1, player2]
    }
def get_current_state(players) -> dict:
    """
    Retrieves the current state of each player, including position, owned roads, 
    money, mortgaged properties, and other status details.

    Args:
        players (list[Player]): List of Player objects in the game.

    Returns:
        dict: A dictionary containing:
            - "players": A list of dictionaries, each with a player's state.
    """
    
    current_state = {
        "players": [{"state": player.get_state()} for player in players]
    }
    return current_state

### Run a Game

In [17]:
game = initialize_game()
player1 = game["players"][0]
player2 = game["players"][1]
list_players = [player1, player2]

# WARNING: KEEP THIS UNDER 10 ROUNDS FOR NOW
# EACH ROUND COSTS US MONEY
STOP_AT_ROUND = 2
idx_count = 0
while not player1.has_lost() and not player2.has_lost() and idx_count < STOP_AT_ROUND:
    for player in list_players:
        # Uncomment this line to run the game if you're sure there are no bugs
        player.play() 
    idx_count += 1

/var/folders/6r/1tc8r8nj3dqg8n_prswrmlpm0000gn/T/ipykernel_765/4015571706.py:122: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  arguments_for = pro_chain.run(


Lawyer FOR Arguments: Ladies and gentlemen of the jury, today, I stand before you to argue in favor of the decision to acquire King's Cross Station in the game of Monopoly. This acquisition is not only a smart financial move but also a strategic decision that positions our client to maximize their chances of winning. Allow me to outline the reasons why purchasing this property is in our best interest.

1. **Strong Return on Investment**: King's Cross Station costs a modest sum of 200. The base rent for landing on this property is 25, which translates to a 12.5% return on investment with just one station involved. But the true value lies in the potential for even greater returns as we expand our holdings. If we acquire more stations in the future, the rent scales dramatically — 50 when two stations are owned, 100 with three, and 200 with four. This exponential increase in potential income makes it a prime target for investment.

2. **Strategic Property Acquisition**: There are four stat

In [26]:
import csv
import os
from datetime import datetime
from typing import Optional

def get_winner(players):
    """
    Determine the winner among players. 
    
    If exactly one player has lost, the other is the winner. 
    If neither has lost by the time we stop, pick the one with the 
    greatest net worth (cash + mortgageable_amount) as the winner.
    Returns:
        winner (Player) or None if somehow there's a tie.
    """
    active_players = [p for p in players if not p.has_lost()]

    if len(active_players) == 1:
        # Exactly one survivor
        return active_players[0]
    elif len(active_players) == 0:
        # All lost simultaneously => no winner
        return None
    else:
        # No one or more than one still in => compare net worth
        # If there's a tie in net worth, we return None
        top_player = None
        top_net_worth = -1
        
        for p in active_players:
            st = p.get_state()
            net_worth = st["cash"] + st["mortgageable_amount"]
            if net_worth > top_net_worth:
                top_player = p
                top_net_worth = net_worth
            elif net_worth == top_net_worth:
                # tie
                top_player = None
        
        return top_player


def run_experiment(
    open_ai_key: str,
    raw_log_filepath: str,
    win_rate_filepath: str,
    max_steps_per_game: int = 10,
    num_games: int = 10,
    reflect_before_vectorstore: bool = False,
    use_default_agents: bool = True,
    primary_model: str = "gpt-4o-mini",
    primary_model_params: dict = {},
    secondary_model: str = "gpt-4o",
    secondary_model_params: dict = {},
) -> None:
    """
    Run an experiment with the given parameters, ensuring ALL data are saved.

    Parameters
    ----------
    open_ai_key : str
        The OpenAI API key.
    raw_log_filepath : str
        Base filepath to save outputs from all LLM completions and vectorstore 
        search results for each game. We'll append `_game{i}_timestamp.log`.
    win_rate_filepath : str
        Base filepath to save CSV of game outcomes (two columns: 
        'raw_log_filepath' and 'custom_agent_won'). We'll append `_timestamp.csv`.
    max_steps_per_game : int, optional
        The maximum number of steps per game.
    num_games : int, optional
        The number of games to play to get the average win rate.
    reflect_before_vectorstore : bool, optional
        Whether to reflect and summarize data before adding past context to 
        vectorstore.
    use_default_agents : bool, optional
        If True, create only one custom agent (the rest are default).
    primary_model : str, optional
        Name of the primary “fast” LLM to use (for lawyer chains).
    primary_model_params : dict, optional
        Key-value pairs for the primary LLM (e.g. {"temperature": 0.7}).
    secondary_model : str, optional
        Name of the secondary “slow” LLM to use (for judge chains).
    secondary_model_params : dict, optional
        Key-value pairs for the secondary LLM (e.g. {"temperature": 0.0}).
    """
    # -------------------------------------------------------------------------
    # 1. Configure your keys/models/logging based on parameters
    # -------------------------------------------------------------------------
    # For demonstration, we’ll just print them. In practice, you can
    # set environment variables or pass the key to your LLM init calls, etc.
    os.environ["OPENAI_API_KEY"] = open_ai_key
    print(f"Using primary model: {primary_model} with params: {primary_model_params}")
    print(f"Using secondary model: {secondary_model} with params: {secondary_model_params}")
    print(f"Reflect before vectorstore? {reflect_before_vectorstore}")

    # Prepare path for CSV that tracks win/loss for each game
    timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")
    final_win_rate_csv = f"{win_rate_filepath}_{timestamp_str}.csv"

    # Ensure the directory exists for the CSV
    os.makedirs(os.path.dirname(final_win_rate_csv), exist_ok=True)

    # -------------------------------------------------------------------------
    # 2. Run multiple games
    # -------------------------------------------------------------------------
    with open(final_win_rate_csv, mode="w", newline="", encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)
        # Write header
        writer.writerow(["raw_log_filepath", "custom_agent_won"])

        for game_index in range(num_games):
            # We’ll log everything into a single .log file for this game
            game_log_file = f"{raw_log_filepath}_game{game_index}_{timestamp_str}.log"
            os.makedirs(os.path.dirname(game_log_file), exist_ok=True)

            # -----------------------------------------------------------------
            # 2A. Initialize a new game
            # -----------------------------------------------------------------
            game_data = initialize_game()
            players = game_data["players"]

            # Optional: If you want to place multiple “CustomPlayer” agents, do so here
            # If use_default_agents = False, you could do a loop of “CustomPlayer(...)” 
            # for all players. But for now, we just rely on the existing initialize_game.

            # -----------------------------------------------------------------
            # 2B. Run up to max_steps_per_game
            # -----------------------------------------------------------------
            step = 0
            while step < max_steps_per_game:
                # Check if the game already ended
                if any(p.has_lost() for p in players):
                    break

                for p in players:
                    if not p.has_lost():
                        # Potentially, you can redirect or capture logs here
                        # For demonstration, we’ll just do a naive open/append
                        with open(game_log_file, "a", encoding="utf-8") as f:
                            f.write(f"\n--- Step {step}, Player: <INSERT NAME> ---\n")
                        # Play a turn
                        p.play()

                step += 1

            # -----------------------------------------------------------------
            # 2C. Determine winner
            # -----------------------------------------------------------------
            winner = get_winner(players)

            # Check if the winner is our custom agent (player1 in this setup)
            # or not. If no winner (tie), we treat as a loss for custom agent.
            custom_agent = players[0]  # "Alice" as in your setup
            custom_agent_won = (winner == custom_agent)

            # Write to CSV
            writer.writerow([game_log_file, str(custom_agent_won)])

            # Optional: Print result
            print(f"Game {game_index} finished. Winner: { "<INSERT NAME>"  if winner else 'None'}")

    print(f"\nAll {num_games} games complete. Results saved to: {final_win_rate_csv}")



In [27]:
run_experiment(
    open_ai_key=openai_key,
    raw_log_filepath="logs/courtroom_raw",
    win_rate_filepath="logs/courtroom_win_rate",
    max_steps_per_game=10,
    num_games=3,
    reflect_before_vectorstore=False,
    primary_model="gpt-4o-mini",
    primary_model_params={"temperature": 0.7},
    secondary_model="gpt-4o",
    secondary_model_params={"temperature": 0.0},
)

Using primary model: gpt-4o-mini with params: {'temperature': 0.7}
Using secondary model: gpt-4o with params: {'temperature': 0.0}
Reflect before vectorstore? False
Lawyer FOR Arguments: Ladies and gentlemen of the Monopoly board, I stand before you to advocate decisively for the purchase of Pall Mall, a sound strategic investment that offers not only immediate benefits but also significant long-term potential for maximally increasing our chances of victory in this game of strategy and negotiation.

**1. Affordable Investment with Promising Yield:**
At a cost of only 140, Pall Mall is an affordable investment given our current cash reserves of 1500. This acquisition represents merely 9.33% of our total cash, allowing us to maintain a healthy cash flow while simultaneously expanding our asset portfolio. The base rent of 10 means this property can yield a modest yet consistent income, which can be critical in the early stages of the game.

**2. Potential for High Returns:**
The real allu

KeyboardInterrupt: 